In [1]:
import openai

In [5]:
from dotenv import dotenv_values

In [9]:
config = dotenv_values('.env')

In [11]:
openai.api_key=config['OPENAI_API_KEY']

In [12]:
import pandas as pd

In [13]:
df = pd.read_csv('sales_data_sample.csv', encoding='cp437')

In [14]:
df.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium


In [15]:
from sqlalchemy import create_engine
from sqlalchemy import text

In [16]:
import sqlalchemy

In [17]:
temporary_db = create_engine('sqlite:///:memory:',echo=True)

In [24]:
with temporary_db.connect() as conn:
    result=conn.execute(text('SELECT * from Sales'))

2023-07-19 10:34:34,504 INFO sqlalchemy.engine.Engine SELECT * from Sales
2023-07-19 10:34:34,504 INFO sqlalchemy.engine.Engine [generated in 0.00139s] ()


In [28]:
def create_definition(db):
    prompt="""
    #Sales({})
    """.format(','.join(str(col)for col in result.keys()))
    return prompt

In [32]:
create_definition(temporary_db)

'\n    #Sales(index,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE)\n    '

In [34]:
def prompt_input():
    nlp_text = input('What do you want to do?: ')
    return nlp_text

In [35]:
def combine(df,prompt_input):
    definition = create_definition(df)
    query=f'Query to answer: {prompt_input()}'
    return definition+query

In [36]:
response=openai.Completion.create(
model='text-davinci-003',
prompt=combine(df,prompt_input),
temperature=0,
max_tokens=150,
frequency_penalty=0,
presence_penalty=0,
stop=['#',';'],
n=1)

What do you want to do?: sum of sales per year accumulated


In [37]:
response

<OpenAIObject text_completion id=cmpl-7dwzzYnykb4CxzNwvPWX9o6pZcZUR at 0x1374c8ca810> JSON: {
  "id": "cmpl-7dwzzYnykb4CxzNwvPWX9o6pZcZUR",
  "object": "text_completion",
  "created": 1689755939,
  "model": "text-davinci-003",
  "choices": [
    {
      "text": "\n    SELECT YEAR_ID, SUM(SALES)\n    FROM Sales\n    GROUP BY YEAR_ID",
      "index": 0,
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 139,
    "completion_tokens": 24,
    "total_tokens": 163
  }
}

In [38]:
def return_sql_query(response):
    query=response['choices'][0]['text']
    if query.startswith(' '):
        query='SELECT'+query
    return query

In [39]:
return_sql_query(response)

'\n    SELECT YEAR_ID, SUM(SALES)\n    FROM Sales\n    GROUP BY YEAR_ID'

In [40]:
with temporary_db.connect() as conn:
    result = conn.execute(text(return_sql_query(response)))

2023-07-19 10:39:15,138 INFO sqlalchemy.engine.Engine 
    SELECT YEAR_ID, SUM(SALES)
    FROM Sales
    GROUP BY YEAR_ID
2023-07-19 10:39:15,140 INFO sqlalchemy.engine.Engine [generated in 0.00100s] ()


In [41]:
result.all()

[(2003, 3516979.540000001), (2004, 4724162.599999997), (2005, 1791486.71)]